In [1]:
# https://stable-baselines3.readthedocs.io/en/master/guide/rl.html
# https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html#a-taxonomy-of-rl-algorithms

# 1. Import dependencies

In [2]:
!pip install stable-baselines3[extra] --quiet

In [3]:
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Load Environment

In [4]:
environment_name = "CartPole-v0"

In [5]:
env = gym.make(environment_name)

In [6]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:12.0
Episode:2 Score:29.0
Episode:3 Score:19.0
Episode:4 Score:23.0
Episode:5 Score:18.0


# Understanding The Environment
https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [7]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

1

In [8]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

array([ 3.7279680e+00, -3.3259783e+38,  2.9364577e-01, -1.7299743e+38],
      dtype=float32)

# 3. Train an RL Model

In [9]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1)

Using cpu device


In [10]:
model.learn(total_timesteps=20000)

-----------------------------
| time/              |      |
|    fps             | 1272 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 880         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009884289 |
|    clip_fraction        | 0.11        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00279     |
|    learning_rate        | 0.0003      |
|    loss                 | 6.77        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0157     |
|    value_loss           | 49.5        |
-----------------------------------------
----------------------------------

# 4. Save and Reload Model

In [11]:
import os
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [12]:
model.save(PPO_path)

In [13]:
del model

In [14]:
model = PPO.load(PPO_path, env=env)

# 4. Evaluation

In [15]:
from stable_baselines3.common.evaluation import evaluate_policy

In [16]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)
env.close()

C:\Users\syeda\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


# 5. Test Model

In [19]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break
env.close()

info [{'TimeLimit.truncated': True, 'terminal_observation': array([-0.62769026, -0.01401242,  0.00954788, -0.08328715], dtype=float32)}]


# 6. Adding a callback to the training Stage

In [22]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [23]:
save_path = os.path.join('Training', 'Saved Models')

In [24]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [25]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [26]:
model = PPO('MlpPolicy', env, verbose = 1)

Using cpu device


In [27]:
model.learn(total_timesteps=20000, callback=eval_callback)

-----------------------------
| time/              |      |
|    fps             | 1032 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 846        |
|    iterations           | 2          |
|    time_elapsed         | 4          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.00916525 |
|    clip_fraction        | 0.0854     |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.687     |
|    explained_variance   | -0.00267   |
|    learning_rate        | 0.0003     |
|    loss                 | 7.61       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0132    |
|    value_loss           | 55.4       |
----------------------------------------
------------------------------------------
| time/  

In [28]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [29]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)
env.close()

# 8. Changing Policies

In [30]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [31]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

Using cpu device


In [32]:
model.learn(total_timesteps=20000, callback=eval_callback)

-----------------------------
| time/              |      |
|    fps             | 865  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 574         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.013925928 |
|    clip_fraction        | 0.197       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.681      |
|    explained_variance   | 0.00169     |
|    learning_rate        | 0.0003      |
|    loss                 | 2.76        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0238     |
|    value_loss           | 19.2        |
-----------------------------------------
----------------------------------

# 9. Using an Alternate Algorithm

In [33]:
from stable_baselines3 import DQN

In [34]:
model = DQN('MlpPolicy', env, verbose = 1)

Using cpu device


In [35]:
model.learn(total_timesteps=20000, callback=eval_callback)

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.969    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 5076     |
|    time_elapsed     | 0        |
|    total_timesteps  | 66       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.923    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 6035     |
|    time_elapsed     | 0        |
|    total_timesteps  | 163      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.889    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 5679     |
|    time_elapsed     | 0        |
|    total_timesteps  | 233      |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 112      |
|    fps              | 6013     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2436     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 116      |
|    fps              | 6013     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2502     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 120      |
|    fps              | 6052     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2591     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 220      |
|    fps              | 6204     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4890     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 224      |
|    fps              | 6209     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4956     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 228      |
|    fps              | 6190     |
|    time_elapsed     | 0        |
|    total_timesteps  | 5022     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 328      |
|    fps              | 6200     |
|    time_elapsed     | 1        |
|    total_timesteps  | 7325     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 332      |
|    fps              | 6190     |
|    time_elapsed     | 1        |
|    total_timesteps  | 7412     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 336      |
|    fps              | 6184     |
|    time_elapsed     | 1        |
|    total_timesteps  | 7498     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 436      |
|    fps              | 6235     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9774     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 440      |
|    fps              | 6236     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9868     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 444      |
|    fps              | 6241     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9982     |
----------------------------------
Eval num_timesteps=10000, episode_reward=8.40 +/- 0.49


----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 536      |
|    fps              | 6146     |
|    time_elapsed     | 1        |
|    total_timesteps  | 11951    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 540      |
|    fps              | 6153     |
|    time_elapsed     | 1        |
|    total_timesteps  | 12043    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 544      |
|    fps              | 6161     |
|    time_elapsed     | 1        |
|    total_timesteps  | 12182    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 644      |
|    fps              | 6158     |
|    time_elapsed     | 2        |
|    total_timesteps  | 14407    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 648      |
|    fps              | 6163     |
|    time_elapsed     | 2        |
|    total_timesteps  | 14543    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 652      |
|    fps              | 6165     |
|    time_elapsed     | 2        |
|    total_timesteps  | 14706    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 752      |
|    fps              | 6171     |
|    time_elapsed     | 2        |
|    total_timesteps  | 16881    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 756      |
|    fps              | 6161     |
|    time_elapsed     | 2        |
|    total_timesteps  | 16948    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 760      |
|    fps              | 6159     |
|    time_elapsed     | 2        |
|    total_timesteps  | 17054    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 860      |
|    fps              | 6242     |
|    time_elapsed     | 3        |
|    total_timesteps  | 19279    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 864      |
|    fps              | 6243     |
|    time_elapsed     | 3        |
|    total_timesteps  | 19348    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 868      |
|    fps              | 6238     |
|    time_elapsed     | 3        |
|    total_timesteps  | 19415    |
----------------------------------
----------------------------------
| rollout/          

In [36]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [37]:
model.save(dqn_path)

In [38]:
model = DQN.load(dqn_path, env=env)

In [39]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)
env.close()